<a href="https://colab.research.google.com/github/financieras/big_data/blob/main/leccion_2_1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lección 2.1.2: Manipulación de datos con Pandas (lectura, limpieza, profiling)

## 1. Pandas: El bisturí del data scientist

Pandas es para los datos lo que un bisturí para un cirujano: **la herramienta precisa que transforma lo complejo en manejable**. No es exageración—el 80% del trabajo en datos pasa por manipular tablas, y Pandas domina este arte.

> **Idea clave:** Pandas convierte datos caóticos en información estructurada lista para análisis.

**¿Por qué Pandas es indispensable?**
- 📊 Maneja datos tabulares de manera intuitiva (DataFrames)
- ⚡ Operaciones vectorizadas: rápidas como C, simples como Python
- 🔧 Integración perfecta con el ecosistema de datos
- 🤝 Comunidad masiva y documentación excelente

**Ejemplo visual de transformación:**

**Antes: Datos desordenados en CSV**
```
nombre,edad,ciudad,salario
Ana,28,Madrid,45000
Juan,32,Barcelona,52000
María,,Valencia,38000
Luis,29,Madrid,41000
```

**Después: DataFrame limpio y estructurado**
| nombre | edad | ciudad    | salario |
|--------|------|-----------|---------|
| Ana    | 28   | Madrid    | 45000   |
| Juan   | 32   | Barcelona | 52000   |
| María  | 29*  | Valencia  | 38000   |
| Luis   | 29   | Madrid    | 41000   |

*Nulo imputado con mediana

---

## 2. Lectura de datos: Tu punto de partida

### **Formatos esenciales y cómo leerlos**

```python
import pandas as pd

# CSV (el caballo de batalla)
df = pd.read_csv('datos.csv', encoding='utf-8')

# Excel (el favorito del negocio)
df = pd.read_excel('reporte.xlsx', sheet_name='Ventas')

# JSON (APIs y web)
df = pd.read_json('datos_api.json')

# Desde SQL
import sqlite3
conn = sqlite3.connect('database.db')
df = pd.read_sql('SELECT * FROM ventas', conn)

# Desde URL directa
df = pd.read_csv('https://ejemplo.com/datos.csv')

# Desde el portapapeles (¡súper útil!)
df = pd.read_clipboard()
```

### **Parámetros que salvan vidas**

```python
df = pd.read_csv('datos_sucios.csv',
                 encoding='latin-1',       # Para caracteres especiales (ñ, á)
                 sep=';',                  # Separador diferente a coma
                 decimal=',',              # Decimales europeos
                 na_values=['NULL', 'N/A', ''],  # Valores faltantes personalizados
                 dtype={'telefono': str},  # Forzar tipo de datos
                 parse_dates=['fecha'],    # Convertir a fecha automáticamente
                 nrows=1000)              # Solo primeras 1000 filas (testing)
```

**Caso real crítico:** Un analista recibe datos de ventas europeos con decimales con coma. Sin `decimal=','`, el precio **1.200,50€** se interpreta como **1.20050€**. ¡Error catastrófico que puede costar millones!

---

## 3. Exploración inicial: Los primeros 5 minutos

**Estos son los comandos que debes ejecutar SIEMPRE:**

```python
# Los 5 comandos esenciales
print(df.shape)          # Dimensiones (filas, columnas)
print(df.info())         # Tipos de datos, memoria, nulos
print(df.head())         # Primeras 5 filas
print(df.describe())     # Estadísticas de columnas numéricas
print(df.columns.tolist())  # Lista de nombres de columnas
```

### **Inspección avanzada**

```python
# Muestra aleatoria (mejor que head() para datasets grandes)
df.sample(10)

# Valores únicos por columna
df.nunique()

# Verificar duplicados
print(f"Duplicados: {df.duplicated().sum()}")

# Memoria utilizada
print(f"Memoria: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
```

---

## 4. Limpieza de datos: De sucio a brillante

### **Detección y manejo de valores faltantes**

```python
# Diagnóstico completo
print("Valores nulos por columna:")
print(df.isnull().sum())
print("\nPorcentaje de nulos:")
print((df.isnull().mean() * 100).round(2))

# Estrategias según el contexto
# 1. Eliminar (cuando son pocos y aleatorios)
df_clean = df.dropna(subset=['columna_critica'])

# 2. Imputar con media/mediana (variables numéricas)
df['edad'] = df['edad'].fillna(df['edad'].median())

# 3. Imputar con moda (variables categóricas)
df['ciudad'] = df['ciudad'].fillna(df['ciudad'].mode()[0])

# 4. Imputar por grupos (más inteligente)
df['precio'] = df.groupby('categoria')['precio'].transform(
    lambda x: x.fillna(x.median())
)

# 5. Forward fill (datos temporales)
df['ventas'] = df['ventas'].fillna(method='ffill')
```

### **Corrección de tipos de datos**

```python
# Los tipos incorrectos son bombas de tiempo
# Conversiones esenciales
df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce')
df['precio'] = pd.to_numeric(df['precio'], errors='coerce')
df['categoria'] = df['categoria'].astype('category')  # Ahorra memoria

# Verificación
print("Tipos después de la corrección:")
print(df.dtypes)
```

### **Limpieza de texto y estandarización**

```python
# Texto inconsistente es un dolor de cabeza común
df['nombre'] = df['nombre'].str.strip().str.title()
df['email'] = df['email'].str.lower()
df['telefono'] = df['telefono'].str.replace(r'[\s-]', '', regex=True)

# Estandarizar categorías
mapeo_ciudades = {'MAD': 'Madrid', 'mad': 'Madrid', 'MADRID': 'Madrid'}
df['ciudad'] = df['ciudad'].map(mapeo_ciudades).fillna(df['ciudad'])
```

### **Eliminación de duplicados**

```python
# Duplicados exactos
df = df.drop_duplicates()

# Duplicados en columnas clave
df = df.drop_duplicates(subset=['id_usuario', 'fecha'])

# Mantener el último registro
df = df.drop_duplicates(subset=['id'], keep='last')

# Ver duplicados antes de eliminar
duplicados = df[df.duplicated(keep=False)]
print(f"Encontrados {len(duplicados)} registros duplicados")
```

---

## 5. Transformaciones esenciales

### **Selección y filtrado**

```python
# Seleccionar columnas
df[['producto', 'precio']]

# Filtrar filas
df[df['precio'] > 100]
df[(df['precio'] > 50) & (df['rating'] >= 4.0)]  # Múltiples condiciones

# Query SQL-like (más legible)
df.query('precio > 100 and categoria == "Ropa"')
```

### **Creación de nuevas columnas**

```python
# Columna simple
df['precio_total'] = df['precio'] * df['cantidad']

# Columna condicional con np.where
import numpy as np
df['descuento'] = np.where(df['precio'] > 100, df['precio'] * 0.10, 0)

# Múltiples condiciones con np.select
condiciones = [
    (df['precio'] < 50),
    (df['precio'] >= 50) & (df['precio'] < 100),
    (df['precio'] >= 100)
]
categorias = ['Básico', 'Medio', 'Premium']
df['segmento'] = np.select(condiciones, categorias)
```

### **Agregaciones y agrupaciones**

```python
# Agrupar y agregar
df.groupby('categoria')['precio'].mean()

# Múltiples agregaciones
df.groupby('categoria').agg({
    'precio': ['mean', 'min', 'max'],
    'cantidad': 'sum'
})

# Pivot tables
df.pivot_table(values='precio',
               index='categoria',
               columns='ciudad',
               aggfunc='mean')
```

---

## 6. Profiling: Conoce a tus datos en profundidad

### **Función de análisis de distribución**

```python
def analizar_distribucion(df, columna):
    """Análisis estadístico completo de una variable"""
    stats = {
        'media': df[columna].mean(),
        'mediana': df[columna].median(),
        'moda': df[columna].mode()[0] if not df[columna].mode().empty else None,
        'std': df[columna].std(),
        'q1': df[columna].quantile(0.25),
        'q3': df[columna].quantile(0.75),
        'iqr': df[columna].quantile(0.75) - df[columna].quantile(0.25),
        'skewness': df[columna].skew(),
        'kurtosis': df[columna].kurtosis()
    }
    return stats

# Uso
stats_ventas = analizar_distribucion(df, 'ventas')
print(stats_ventas)
```

### **Función de reporte de calidad**

```python
def reporte_calidad(df):
    """Reporte completo de calidad de datos"""
    calidad = pd.DataFrame({
        'tipo_dato': df.dtypes,
        'valores_no_nulos': df.count(),
        'valores_nulos': df.isnull().sum(),
        'porcentaje_nulos': (df.isnull().mean() * 100).round(2),
        'valores_unicos': df.nunique(),
        'memoria_mb': (df.memory_usage(deep=True) / 1024**2).round(2)
    })
    return calidad

# Reporte ejecutivo
print(reporte_calidad(df))
```

### **Profiling automatizado (opcional)**

```python
# Pandas Profiling - Reporte HTML completo
from pandas_profiling import ProfileReport

profile = ProfileReport(df, title="Análisis de Ventas")
profile.to_file("reporte_eda.html")

# Incluye: estadísticas, distribuciones, correlaciones,
# valores faltantes, duplicados, alertas de calidad
```

---

## 7. Caso práctico completo: E-commerce

**Problema:** Dataset de 10,000 pedidos con múltiples problemas de calidad.

```python
import pandas as pd
import numpy as np

# 1. LECTURA con parámetros específicos
pedidos = pd.read_csv('pedidos_ecommerce.csv',
                     sep=';',
                     decimal=',',
                     parse_dates=['fecha_pedido', 'fecha_envio'],
                     encoding='latin-1')

# 2. DIAGNÓSTICO INICIAL
print("=== DIAGNÓSTICO INICIAL ===")
print(f"Shape: {pedidos.shape}")
print(f"Memoria: {pedidos.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
print("\nValores nulos:")
print(pedidos.isnull().sum())
print(f"\nDuplicados: {pedidos.duplicated().sum()}")

# 3. LIMPIEZA SISTEMÁTICA

# Corrección de precios negativos o cero
pedidos = pedidos[pedidos['precio'] > 0]

# Imputación de categorías faltantes
pedidos['categoria'] = pedidos['categoria'].fillna('OTROS')

# Estandarización de ciudades
pedidos['ciudad'] = pedidos['ciudad'].str.upper().str.strip()

# Eliminación de duplicados
pedidos = pedidos.drop_duplicates(subset=['id_pedido'])

# Corrección de tipos
pedidos['telefono'] = pedidos['telefono'].astype(str)

# 4. FEATURE ENGINEERING BÁSICO
pedidos['dias_entrega'] = (
    pedidos['fecha_envio'] - pedidos['fecha_pedido']
).dt.days
pedidos['valor_total'] = pedidos['precio'] * pedidos['cantidad']
pedidos['mes'] = pedidos['fecha_pedido'].dt.month
pedidos['trimestre'] = pedidos['fecha_pedido'].dt.quarter

# 5. VALIDACIÓN
assert pedidos['precio'].min() > 0, "Hay precios negativos"
assert pedidos['dias_entrega'].min() >= 0, "Fechas inconsistentes"

# 6. PROFILING FINAL
print("\n=== RESULTADO FINAL ===")
print(f"Shape final: {pedidos.shape}")
print(reporte_calidad(pedidos))

# 7. GUARDAR
pedidos.to_csv('pedidos_limpio.csv', index=False)
pedidos.to_parquet('pedidos_limpio.parquet')  # Más eficiente
```

**Resultado cuantificado:**
- **Antes:** 10,000 filas, 15% de nulos, tipos incorrectos, 150 duplicados
- **Después:** 9,850 filas limpias, 0% nulos en campos críticos, 0 duplicados
- **Tiempo de limpieza:** 10 minutos vs 2+ horas manual

---

## 8. Funciones mágicas para el día a día

### **Función de limpieza express**

```python
def limpieza_express(df):
    """Limpieza rápida para datasets comunes"""
    df_clean = df.copy()
    
    # Eliminar duplicados
    df_clean = df_clean.drop_duplicates()
    
    # Limpiar columnas de texto
    text_cols = df_clean.select_dtypes(include=['object']).columns
    for col in text_cols:
        df_clean[col] = df_clean[col].str.strip().str.lower()
    
    return df_clean

# Uso rápido
df_limpio = limpieza_express(df_original)
```

### **Pipeline completo de preparación**

```python
def preparar_para_eda(archivo):
    """Pipeline completo: datos listos para EDA"""
    # 1. Lectura
    df = pd.read_csv(archivo)
    
    # 2. Limpieza básica
    df = limpieza_express(df)
    
    # 3. Profiling inicial
    reporte = reporte_calidad(df)
    
    return df, reporte

# Uso en proyecto real
datos_limpios, diagnostico = preparar_para_eda('datos_brutos.csv')
print(diagnostico)
```

---

## 9. Buenas prácticas y errores comunes

### ✅ **Haz esto:**

```python
# Trabajar con copias para no modificar datos originales
df_clean = df.copy()

# Documentar cada transformación
# NOTA: Imputamos nulos en edad con mediana (distribución sesgada)
df['edad'] = df['edad'].fillna(df['edad'].median())

# Validar resultados con assertions
assert df['precio'].min() >= 0, "ERROR: Hay precios negativos"
assert df.isnull().sum().sum() == 0, "ERROR: Quedan valores nulos"

# Verificar después de operaciones críticas
print(f"Filas antes: {len(df_original)}")
df_clean = df_original.dropna()
print(f"Filas después: {len(df_clean)}")
```

### ❌ **Evita esto:**

```python
# ❌ Modificar el DataFrame original directamente
df = df.dropna()  # ¡Peligro! Pierdes datos originales

# ❌ Usar bucles cuando hay vectorización
for i in range(len(df)):
    df.loc[i, 'nueva'] = df.loc[i, 'col1'] * 2  # LENTO

# ✅ Usa operaciones vectorizadas
df['nueva'] = df['col1'] * 2  # RÁPIDO

# ❌ Ignorar los warnings
# Investiga siempre SettingWithCopyWarning y FutureWarning
```

### **Checklist de manipulación**

- [ ] Datos leídos correctamente (encoding, separadores, tipos)
- [ ] Exploración inicial completada (`info()`, `describe()`)
- [ ] Valores nulos identificados y tratados
- [ ] Duplicados eliminados o justificados
- [ ] Texto estandarizado y limpio
- [ ] Tipos de datos correctos por columna
- [ ] Outliers identificados y validados
- [ ] Transformaciones documentadas con comentarios
- [ ] Validaciones con assertions ejecutadas
- [ ] Dataset guardado en formato eficiente

---

## 10. Resumen

**Pandas es:**
- ✅ El **estándar** para manipulación de datos en Python
- ✅ **Versátil:** Lee cualquier formato (CSV, Excel, JSON, SQL)
- ✅ **Potente:** Limpieza, transformación, agregación
- ✅ **Rápido:** Operaciones vectorizadas

**Flujo típico de trabajo:**
```
1. Leer → read_csv(), read_excel()
2. Explorar → info(), describe(), head()
3. Limpiar → Nulos, duplicados, tipos
4. Transformar → Filtrar, agrupar, nuevas columnas
5. Validar → Assertions, profiling
6. Guardar → to_csv(), to_parquet()
```

**Las 3 operaciones más usadas:**
1. `df[df['columna'] > valor]` → Filtrar
2. `df.groupby('categoria')['valor'].mean()` → Agrupar
3. `df['nueva'] = df['col1'] * df['col2']` → Crear columnas

> **Conclusión:** Dominar Pandas es dominar el 80% del trabajo diario de un Data Analyst. Invierte tiempo en aprenderlo bien—cada minuto pagará dividendos exponenciales.

---

## 11. Referencias

### Vídeos
- [Complete Pandas Tutorial](https://youtu.be/vmEHCJofslg) - Tutorial completo
- [Data Cleaning with Pandas](https://youtu.be/example2) - Limpieza práctica
- [Pandas Best Practices](https://youtu.be/example3) - Tips avanzados

### Lecturas
- [Pandas Documentation](https://pandas.pydata.org/docs/) - Documentación oficial
- [10 Minutes to Pandas](https://pandas.pydata.org/docs/user_guide/10min.html) - Guía rápida
- [Pandas Cheat Sheet](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf) - Referencia rápida
